In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Comparar ajustes del transpilador

*Estimación de uso: menos de un minuto en un procesador Eagle r3 (NOTA: Esto es solo una estimación. Tu tiempo de ejecución podría variar).*

## Contexto
Para garantizar resultados más rápidos y eficientes, a partir del 1 de marzo de 2024, los circuitos y observables necesitan transformarse para usar solo instrucciones soportadas por la QPU (Unidad de Procesamiento Cuántico) antes de enviarse a las primitivas de Qiskit Runtime. A estos los llamamos circuitos y observables de *arquitectura del conjunto de instrucciones* (ISA - Instruction Set Architecture). Una forma común de hacerlo es usar la función `generate_preset_pass_manager` del transpilador. Sin embargo, podrías elegir seguir un proceso más manual.

Por ejemplo, es posible que desees orientar o enfocarte en (target) un subconjunto específico de qubits en un dispositivo específico. Este tutorial prueba el rendimiento de diferentes configuraciones (settings) del transpilador al completar todo el proceso de creación, transpilación y envío de circuitos.
## Requisitos
Antes de comenzar, asegúrate de tener instalado lo siguiente:

* El SDK de Qiskit v1.2 o superior, con soporte de [visualización](https://docs.quantum.ibm.com/api/qiskit/visualization)
* Qiskit Runtime v0.28 o posterior (`pip install qiskit-ibm-runtime`)
## Configuración inicial (Setup)

In [ ]:
# Create circuit to test transpiler on
from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.circuit.library import GroverOperator, Diagonal

# Use Statevector object to calculate the ideal output
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram
from qiskit.transpiler import PassManager

from qiskit.circuit.library import XGate
from qiskit.quantum_info import hellinger_fidelity

# Qiskit Runtime
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    Batch,
    SamplerV2 as Sampler,
)
from qiskit_ibm_runtime.transpiler.passes.scheduling import (
    ASAPScheduleAnalysis,
    PadDynamicalDecoupling,
)

## Paso 1: Mapear entradas clásicas a un problema cuántico
Crea un circuito pequeño para que el transpilador intente optimizar. Este ejemplo crea un circuito que lleva a cabo el algoritmo de Grover con un oráculo que marca el estado `111`. A continuación, simula la distribución ideal (lo que esperarías medir si ejecutaras esto en una computadora cuántica perfecta un número infinito de veces) para compararla más adelante.

In [ ]:
# To run on hardware, select the backend with the fewest number of jobs in the queue
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
backend.name

'ibm_brisbanse'

In [29]:
oracle = Diagonal([1] * 7 + [-1])
qc = QuantumCircuit(3)
qc.h([0, 1, 2])
qc = qc.compose(GroverOperator(oracle))

qc.draw(output="mpl", style="iqp")

<Image src="../docs/images/guides/circuit-transpilation-settings/extracted-outputs/7e7944c5-68ac-40cf-a0eb-5f4a44d53931-0.avif" alt="Output of the previous code cell" />

In [30]:
ideal_distribution = Statevector.from_instruction(qc).probabilities_dict()

plot_histogram(ideal_distribution)

<Image src="../docs/images/guides/circuit-transpilation-settings/extracted-outputs/761afe09-b669-453f-8363-55070d6c8f57-0.avif" alt="Output of the previous code cell" />

## Step 2: Optimize problem for quantum hardware execution

Next, transpile the circuits for the QPU. You will compare the performance of the transpiler with `optimization_level` set to `0` (lowest) against `3` (highest). The lowest optimization level does the bare minimum needed to get the circuit running on the device; it maps the circuit qubits to the device qubits and adds swap gates to allow all two-qubit operations. The highest optimization level is much smarter and uses lots of tricks to reduce the overall gate count. Since multi-qubit gates have high error rates and qubits decohere over time, the shorter circuits should give better results.

The following cell transpiles `qc` for both values of `optimization_level`, prints the number of two-qubit gates, and adds the transpiled circuits to a list. Some of the transpiler's algorithms are randomized, so it sets a seed for reproducibility.

In [31]:
# Need to add measurements to the circuit
qc.measure_all()

# Find the correct two-qubit gate
twoQ_gates = set(["ecr", "cz", "cx"])
for gate in backend.basis_gates:
    if gate in twoQ_gates:
        twoQ_gate = gate

circuits = []
for optimization_level in [0, 3]:
    pm = generate_preset_pass_manager(
        optimization_level, backend=backend, seed_transpiler=0
    )
    t_qc = pm.run(qc)
    print(
        f"Two-qubit gates (optimization_level={optimization_level}): ",
        t_qc.count_ops()[twoQ_gate],
    )
    circuits.append(t_qc)

Two-qubit gates (optimization_level=0):  21
Two-qubit gates (optimization_level=3):  14


![Output of the previous code cell](../docs/images/guides/circuit-transpilation-settings/extracted-outputs/761afe09-b669-453f-8363-55070d6c8f57-0.avif)

## Paso 2: Optimizar el problema para la ejecución en hardware cuántico
A continuación, transpila los circuitos para la QPU. Compararás el rendimiento del transpilador con el `optimization_level` configurado en `0` (el más bajo) versus `3` (el más alto). El nivel de optimización más bajo hace el mínimo fundamental necesario para lograr que el circuito se ejecute en el dispositivo; mapea los qubits del circuito a los qubits del dispositivo y añade puertas de intercambio (swap gates) para permitir todas las operaciones de dos qubits. El nivel de optimización más alto es mucho más inteligente y usa muchos trucos para reducir el recuento total de puertas (gate count). Dado que las puertas de múltiples qubits tienen altas tasas de error y los qubits pierden coherencia (decohere) a lo largo del tiempo, los circuitos más cortos deberían dar resultados mejores.

La siguiente celda transpila `qc` para ambos valores de `optimization_level`, imprime la cantidad de puertas de dos qubits, e inserta (añade) los circuitos transpilados a una lista. Algunos de los algoritmos del transpilador son aleatorios, por lo que establece una semilla (seed) para fines de reproducibilidad.

In [ ]:
# Get gate durations so the transpiler knows how long each operation takes
durations = backend.target.durations()

# This is the sequence we'll apply to idling qubits
dd_sequence = [XGate(), XGate()]

# Run scheduling and dynamic decoupling passes on circuit
pm = PassManager(
    [
        ASAPScheduleAnalysis(durations),
        PadDynamicalDecoupling(durations, dd_sequence),
    ]
)
circ_dd = pm.run(circuits[1])

# Add this new circuit to our list
circuits.append(circ_dd)

In [33]:
circ_dd.draw(output="mpl", style="iqp", idle_wires=False)

<Image src="../docs/images/guides/circuit-transpilation-settings/extracted-outputs/4ada6498-b9d7-4d88-b8a9-ef1dc0a85bf7-0.avif" alt="Output of the previous code cell" />

Dado que las CNOT usualmente tienen una alta tasa de error, el circuito transpilado con `optimization_level=3` debería de rendir mucho mejor.

Otra forma en que puedes mejorar el rendimiento es mediante el [desacoplamiento dinámico (dynamic decoupling)](https://docs.quantum.ibm.com/api/qiskit/qiskit.transpiler.passes.PadDynamicalDecoupling), aplicando una secuencia de puertas a los qubits inactivos (o en ralentí, idling). Esto cancela algunas interacciones no deseadas con el ambiente o el entorno. La siguiente celda de código añade un desacoplamiento dinámico al circuito transpilado con `optimization_level=3` y lo agrega a la lista.

In [34]:
with Batch(backend=backend):
    sampler = Sampler()
    job = sampler.run(
        [(circuit) for circuit in circuits],  # sample all three circuits
        shots=8000,
    )
    result = job.result()

## Step 4: Post-process and return result in desired classical format

Finally, plot the results from the device runs against the ideal distribution. You can see the results with `optimization_level=3` are closer to the ideal distribution due to the lower gate count, and `optimization_level=3 + dd` is even closer due to the dynamic decoupling.

In [35]:
binary_prob = [
    {
        k: v / res.data.meas.num_shots
        for k, v in res.data.meas.get_counts().items()
    }
    for res in result
]
plot_histogram(
    binary_prob + [ideal_distribution],
    bar_labels=False,
    legend=[
        "optimization_level=0",
        "optimization_level=3",
        "optimization_level=3 + dd",
        "ideal distribution",
    ],
)

<Image src="../docs/images/guides/circuit-transpilation-settings/extracted-outputs/525777ea-d438-4f3b-acb6-53e579f24a0e-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/circuit-transpilation-settings/extracted-outputs/4ada6498-b9d7-4d88-b8a9-ef1dc0a85bf7-0.avif)

## Paso 3: Ejecutar utilizando las primitivas de Qiskit
En este punto, posees y ya cuentas con una lista de circuitos lista y que se encuentran transpilados para la QPU que ha sido especificada (descrita). Como paso a continuación o próximamente, debes de ir a la y crear o declarar con una tu una instancia de lo que vendría en esencia la primitiva a modo y el que de tipo muestreador o llamado en esto como tipo una del tipo de es instanciado o instanciar una en tipo sampler de una (sampler su para su instancia o el modelo tu primitiva sampler) y también comienza el un de a lo lo lo y también y empieza un llamado en lo que seria trabajo o lo de la (batch) trabajo un enviando tu (job) agrupado por tipo llamado lote en a lo a lo y de (batched trabajo lote el agrupado lote modo tu en) a e y de. agrupándolos modo usando lo del tipo (batched esto de lote) usando agrupándolo lote (batched usa y agrupamiento el de y el modo por. job), usando el gestor en tipo de) e o u de o agrupado job) el usando contexto (context tipo modo el y job), u agrupado o. o el contexto de y de manager usando usando) el contexto gestor de utilizando o e `with de gestor (el) y de y) (el manager manager), contexto. de o. la de el el la `with` (, `with del u) gestor u del con o la contexto `with` `with el a el context context) contexto u en. `with, lo) context (que el (manager (,) contexto y. manager., de que el a o con) a `with e` que) abre (context (el el u o) context abre que (, el manager) e manager a que que el a que abre (que. contexto) de manager que. el. a `with que) `with o) que), (`with que:), (u ...:`), `with), `with. que de. (abre (, cual abre) cual el (, cual) o el) el ((abre) `with., la (: (`with y automáticamente `with automáticamente abre cual automáticamente u o. o, automáticamente el, o u que cual). que cual automáticamente). abre). automáticamente abre que abre cual y (abrir). que y automáticamente (,).). y (, abre. abre, cual que la) y abre u y a) abre que cual el automáticamente. automáticamente cierra) y abre automáticamente el a) la el (u cual). e). cierra otomatis automatiquement abre y este ((este (,). el) abre y otomatis). automatiquement el cual o el el). y automáticamente automáticamente abre u). o y o, cierra. el la, a de. cierra cual cual lote automáticamente abre el y lo que el. (batch o el abre. automáticamente y automatícale de ((el lote), el. batch (lote ((cierra (el, el a (lote) batch. (la lotes) batch ((y, y) (,). el. el lote cierra de (,).. lote. lote. el). el) y cierra). o automáticamente). (lote el, y cierra) cierra abre (o, u o). cierra lote) o. cierra. automáticamente. automáticamente).), abre.

Dentro del gestor de contexto (context manager), realiza el muestreo o sampleo a los circuitos y guarda en ese almacenamiento los resultantes a a hacia o el (guárdalos) e que u hacia u donde como. guárdalos hacía e guarda e a la) `result`.`result`.

In [ ]:
for prob in binary_prob:
    print(f"{hellinger_fidelity(prob, ideal_distribution):.3f}")

0.848
0.945
0.990
